# Model Training for Customer Churn prediction

## Feature Engineering

During data analysis, we found that our data has no missing values, duplicate rows or outliers. Hence, we don't have to handle these issues.

In [1]:
import pandas as pd
import numpy as np

In [2]:
original_df = pd.read_excel("../dataset/customer_churn_large_dataset.xlsx")
original_df.head()
df = original_df

df = df.drop(columns=["Name", "CustomerID"], errors="ignore")

df.head()

,Age,Gender,Location,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB,Churn
0,63,Male,Los Angeles,17,73.36,236,0
1,62,Female,New York,1,48.76,172,0
2,24,Female,Los Angeles,5,85.47,460,0
3,36,Female,Miami,3,97.94,297,1
4,46,Female,Miami,19,58.14,266,0


In [3]:
df.Gender = df.Gender.apply(lambda x: 1 if x=="Male" else 0)
df = df.rename(columns={"Gender": "is_male"})
df.head()

,Age,is_male,Location,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB,Churn
0,63,1,Los Angeles,17,73.36,236,0
1,62,0,New York,1,48.76,172,0
2,24,0,Los Angeles,5,85.47,460,0
3,36,0,Miami,3,97.94,297,1
4,46,0,Miami,19,58.14,266,0


In [4]:
df.Location = df.Location.apply(lambda x: x.replace(" ", "_").lower())

df = pd.get_dummies(df, "", "", columns=["Location"], dtype=int)
df.head()

,Age,is_male,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB,Churn,chicago,houston,los_angeles,miami,new_york
0,63,1,17,73.36,236,0,0,0,1,0,0
1,62,0,1,48.76,172,0,0,0,0,0,1
2,24,0,5,85.47,460,0,0,0,1,0,0
3,36,0,3,97.94,297,1,0,0,0,1,0
4,46,0,19,58.14,266,0,0,0,0,1,0


In [5]:
df = df[["Age", "Subscription_Length_Months", "Monthly_Bill", "Total_Usage_GB",
         "is_male", "chicago", "houston", "los_angeles", "miami", "new_york",
         "Churn"]]

df.head()

,Age,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB,is_male,chicago,houston,los_angeles,miami,new_york,Churn
0,63,17,73.36,236,1,0,0,1,0,0,0
1,62,1,48.76,172,0,0,0,0,0,1,0
2,24,5,85.47,460,0,0,0,1,0,0,0
3,36,3,97.94,297,0,0,0,0,1,0,1
4,46,19,58.14,266,0,0,0,0,1,0,0


In [6]:
df_arr = df.to_numpy(np.float32)
df_arr

array([[63.  , 17.  , 73.36, ...,  0.  ,  0.  ,  0.  ],
       [62.  ,  1.  , 48.76, ...,  0.  ,  1.  ,  0.  ],
       [24.  ,  5.  , 85.47, ...,  0.  ,  0.  ,  0.  ],
       ...,
       [64.  , 17.  , 96.11, ...,  0.  ,  0.  ,  1.  ],
       [51.  , 20.  , 49.25, ...,  0.  ,  1.  ,  1.  ],
       [27.  , 19.  , 76.57, ...,  0.  ,  0.  ,  1.  ]], dtype=float32)

In [7]:
X = df_arr[:, :-1]
Y = df_arr[:, -1]

print("Features:\n", X)
print("\nTarget:\n", Y)

Features:
 [[63.   17.   73.36 ...  1.    0.    0.  ]
 [62.    1.   48.76 ...  0.    0.    1.  ]
 [24.    5.   85.47 ...  1.    0.    0.  ]
 ...
 [64.   17.   96.11 ...  0.    0.    0.  ]
 [51.   20.   49.25 ...  0.    0.    1.  ]
 [27.   19.   76.57 ...  1.    0.    0.  ]]

Target:
 [0. 0. 0. ... 1. 1. 1.]


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=8623)

print("Training data shape:", X_train.shape, Y_train.shape)
print("Testing data shape", X_test.shape, Y_test.shape)

Training data shape: (90000, 10) (90000,)
Testing data shape (10000, 10) (10000,)


In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler((-1,1))
X_train[:,:4] = scaler.fit_transform(X_train[:,:4])
X_test[:,:4] = scaler.transform(X_test[:,:4])

print("X_train:\n", X_train[:5,:5])
print("\nX_test:\n", X_test[:5,:5])

X_train:
 [[-0.96153843 -0.3043478  -0.21142864 -0.12        0.        ]
 [-0.8076923  -0.91304344 -0.96971434 -0.29777777  0.        ]
 [ 0.26923084 -0.7391304   0.13485718  0.80444443  1.        ]
 [-0.7307692   1.         -0.41828573 -0.59999996  1.        ]
 [ 0.7307694  -0.6521739   0.7231426  -0.05777776  1.        ]]

X_test:
 [[-0.6923077   0.652174   -0.27200007 -0.38222218  1.        ]
 [ 0.15384626  0.7391305  -0.4545715  -0.5155555   0.        ]
 [ 0.76923084  0.39130437  0.26885724 -0.42666662  1.        ]
 [ 0.46153855 -0.47826082 -0.6420001  -0.12444437  0.        ]
 [-1.          0.30434787 -0.61371434 -0.09333324  0.        ]]


## Model Training

This is a basic model training program. It is written to quickly code and check the model training function.  
Actual model training will be done using training pipeline script in 'model_trainer' because full training takes a lot of time due to the size of the dataset.

In [10]:
# Models for selection
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

models = {
    "LR": {
        "model": LogisticRegression(random_state=456),
        "params": {
            "penalty": ['l2', None],
            "C": [0.01, 0.1, 0.5, 1]
        }
    },
         
    "Random Forest": {
        "model": RandomForestClassifier(random_state=350),
        "params":{
            "n_estimators": [10, 20],
            "max_depth": [10, 20]
        }
    },
    
    "MLP": {
        "model": MLPClassifier(max_iter=10, random_state=651),
        "params": {
            "hidden_layer_sizes": [
                [100],
                [100, 10]
            ],
            "alpha": [0.0001, 0.01, 0.1, 1],
            "learning_rate_init": [0.001, 0.1, 1],
            "early_stopping": [True]
        }
    },
    
    "XgBoost": {
        "model": XGBClassifier(objective="binary:hinge", random_state=342),
        "params":{
            "n_estimators": [10, 20],
            "learning_rate": [0.01, 0.3]
        }
    }
}

In [11]:
from sklearn.model_selection import GridSearchCV

# GridSearchCV performs cross-validation on a model for each combination of hyperparameters
       
def search_and_train(X, Y, models: dict):
    "Returns best trained model and a dataframe of model performance report"
    
    best_score = None
    best_model = None
    report = {"Model": [], "best_params": [], "accuracy": [],
              "precision": [], "recall": [], "f1": [], "roc_auc": []}

    for name, m in models.items():
        gscv = GridSearchCV(m["model"], m["params"], 
                            scoring=("accuracy", "precision", "recall", "f1", "roc_auc"),
                            refit="recall",
                            verbose=1)
        gscv.fit(X, Y)
        
        mean_test_recall = gscv.cv_results_["mean_test_recall"][gscv.best_index_]
        assert mean_test_recall == gscv.best_score_, \
            f"Best Scores are different: {mean_test_recall} and {gscv.best_score_}"
        
        report["Model"].append(name)
        report["best_params"].append(gscv.best_params_)
        report["recall"].append(gscv.best_score_)
        report["accuracy"].append(gscv.cv_results_["mean_test_accuracy"][gscv.best_index_])
        report["precision"].append(gscv.cv_results_["mean_test_precision"][gscv.best_index_])
        report["f1"].append(gscv.cv_results_["mean_test_f1"][gscv.best_index_])
        report["roc_auc"].append(gscv.cv_results_["mean_test_roc_auc"][gscv.best_index_])
        
        if best_score is None or gscv.best_score_ > best_score:
            best_score = gscv.best_score_
            best_model = gscv.best_estimator_
    
    return best_model, pd.DataFrame(report)

In [12]:
model, report = search_and_train(X_train, Y_train, models)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


e:\Documents\Chaitanya\Git Repositories\Sunbase_Customer_Churn\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
e:\Documents\Chaitanya\Git Repositories\Sunbase_Customer_Churn\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
e:\Documents\Chaitanya\Git Repositories\Sunbase_Customer_Churn\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
e:\Documents\Chaitanya\Git Repositories\Sunbase_Customer_Churn\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
e:\Documents\Chaitanya\Git Repositories\Sunbase_Customer_Churn\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits


e:\Documents\Chaitanya\Git Repositories\Sunbase_Customer_Churn\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
e:\Documents\Chaitanya\Git Repositories\Sunbase_Customer_Churn\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
e:\Documents\Chaitanya\Git Repositories\Sunbase_Customer_Churn\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
e:\Documents\Chaitanya\Git Repositories\Sunbase_Customer_Churn\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Opt

Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [13]:
report

,Model,best_params,accuracy,precision,recall,f1,roc_auc
0,LR,"{'C': 0.01, 'penalty': None}",0.500089,0.498855,0.422673,0.457145,0.500676
1,Random Forest,"{'max_depth': 20, 'n_estimators': 10}",0.501156,0.500051,0.493942,0.496973,0.500038
2,MLP,"{'alpha': 0.01, 'early_stopping': True, 'hidde...",0.499156,0.498093,0.612606,0.535529,0.500320
3,XgBoost,"{'learning_rate': 0.01, 'n_estimators': 10}",0.498889,0.498889,1.000000,0.665678,0.500000


In [14]:
import joblib as jl
import os

os.makedirs("../artifacts", exist_ok=True)
jl.dump(model, "../artifacts/model.jlb")

['../artifacts/model.jlb']

# Model Evaluation

In [15]:
import joblib as jl

model = jl.load("../artifacts/model.jlb")
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=10, n_jobs=None,
              num_parallel_tree=None, objective='binary:hinge', ...)